信用付

In [59]:
# 接口地址
test_url = "https://test.lianlianpay-inc.com/mpay-openapi"
# 支付申请
pay_apply_api = "/v1/creditpay/stagecode/apply"
# 私钥
private_key= "MIICdwIBADANBgkqhkiG9w0BAQEFAASCAmEwggJdAgEAAoGBAOilN4tR7HpNYvSBra/DzebemoAiGtGeaxa+qebx/O2YAdUFPI+xTKTX2ETyqSzGfbxXpmSax7tXOdoa3uyaFnhKRGRvLdq1kTSTu7q5s6gTryxVH2m62Py8Pw0sKcuuV0CxtxkrxUzGQN+QSxf+TyNAv5rYi/ayvsDgWdB3cRqbAgMBAAECgYEAj02d/jqTcO6UQspSY484GLsL7luTq4Vqr5L4cyKiSvQ0RLQ6DsUG0g+Gz0muPb9ymf5fp17UIyjioN+ma5WquncHGm6ElIuRv2jYbGOnl9q2cMyNsAZCiSWfR++op+6UZbzpoNDiYzeKbNUz6L1fJjzCt52w/RbkDncJd2mVDRkCQQD/Uz3QnrWfCeWmBbsAZVoM57n01k7hyLWmDMYoKh8vnzKjrWScDkaQ6qGTbPVL3x0EBoxgb/smnT6/A5XyB9bvAkEA6UKhP1KLi/ImaLFUgLvEvmbUrpzY2I1+jgdsoj9Bm4a8K+KROsnNAIvRsKNgJPWd64uuQntUFPKkcyfBV1MXFQJBAJGs3Mf6xYVIEE75VgiTyx0x2VdoLvmDmqBzCVxBLCnvmuToOU8QlhJ4zFdhA1OWqOdzFQSw34rYjMRPN24wKuECQEqpYhVzpWkA9BxUjli6QUo0feT6HUqLV7O8WqBAIQ7X/IkLdzLa/vwqxM6GLLMHzylixz9OXGZsGAkn83GxDdUCQA9+pQOitY0WranUHeZFKWAHZszSjtbe6wDAdiKdXCfig0/rOdxAODCbQrQs7PYy1ed8DuVQlHPwRGtokVGHATU="
# 连连公钥
ll_public_key= "MIGfMA0GCSqGSIb3DQEBAQUAA4GNADCBiQKBgQCSS/DiwdCf/aZsxxcacDnooGph3d2JOj5GXWi+q3gznZauZjkNP8SKl3J2liP0O6rU/Y/29+IUe+GTMhMOFJuZm1htAtKiu5ekW0GlBMWxf4FPkYlQkPE0FtaoMP3gYfh+OwI+fIRrpW3ySn3mScnc6Z700nU/VYrRkfcSCbSnRwIDAQAB"
# 商户号
mch_id = '302112250000006404'

encoding = 'utf-8'


In [56]:
from Crypto.PublicKey import RSA
from Crypto.Signature import PKCS1_v1_5
 
import hashlib
import base64
import requests
import json
from time import strftime, localtime

In [49]:
# 测试接口地址
test_url = "http://localhost:8085/"

签名工具

In [5]:

def my_hash(data):
   return hashlib.md5(data.encode('utf-8')).hexdigest()
   

In [6]:
# 数据签名

def rsa_sign(data):
    pri_key = RSA.importKey(base64.b64decode(private_key))
    signer = PKCS1_v1_5.new(pri_key)
    hash_obj = my_hash(data)
    signature = base64.b64encode(signer.sign(hash_obj))
    return signature

# sign_res = rsa_sign('xxdda')
# print(sign_res.decode('utf-8'))


In [60]:


def pay_apply():
    url = test_url + pay_apply_api

    data = {
        "mch_id": mch_id,
        "user_id": "CCTEST20210906001",
        "user_ip": "192.168.1.1",
        "txn_seqno": 'CSP_A'+strftime('%Y%m%d%H%M%S', localtime()),
        "txn_time": strftime('%Y%m%d%H%M%S', localtime()),
        "order_amount": '1000',
        "order_info": 'xxdda py aa',
        "notify_url": "http://localtime.com",
        "pay_expire": "60",
        "pay_type":'STAGE_CODE_PAY',
        "stage_info":{
            "stage_num":'6'
        }
    }

    headers = {
        "timestamp": strftime('%Y%m%d%H%M%S', localtime()),
        "Signature-Data": rsa_sign(hashlib.md5(json.dumps(data).encode('utf-8')).hexdigest()).decode("utf-8"),
        "Signature-Type": "RSA",
        "mch_id": mch_id,
        "Content-Type": "application/json;charset=UTF-8"
    }

    r = requests.post(url=url, data=json.dumps(data), headers=headers)
    status = r.status_code
    print(r.json())


pay_apply()


{'ret_code': '0000', 'ret_msg': '交易成功', 'mch_id': '302112250000006404', 'txn_time': '20221122173914', 'txn_seqno': 'CSP_A20221122173914', 'platform_txno': '2022112246767586', 'order_amount': '1000.00', 'payload': '{"gateway_url":"http://101.231.114.212:10002/aps/dynamic/loading.html#loading?id=740531506f32ba7e90c8759a8d1efe9ad8cc8c915cb97cad1394cdcf3d6dd0d0"}'}


In [46]:
print(strftime('%Y%m%d%H%M%S',localtime()))

from mylib import sign

sign.my_hash111("xxxxxxxx")



20221122171758
